In [ ]:
import altair as alt
import pandas as pd

- https://altair-viz.github.io/user_guide/configuration.html
- https://qualar.apambiente.pt/node/metodo-calculo-indices
- https://github.com/altair-viz/altair/issues/2176#issuecomment-633679510
- https://github.com/altair-viz/altair/issues/1768#issuecomment-549370312
- https://altair-viz.github.io/user_guide/transform/impute.html
- https://stackoverflow.com/a/15705958
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.transform.html
- https://tailwindcss.com/docs/font-size

In [ ]:
DATA: str = "https://raw.githubusercontent.com/dssgPT/Plotting-Good-DSSG/main/desafios/002_Qualidade_do_Ar_em_Portugal_EEA/qualidade_do_ar.csv"

In [ ]:
df = pd.read_csv(DATA, index_col=0)
df["cidade"] = df["cidade"].str.strip()

In [ ]:
min_year = df["ano"].min()
max_year = df["ano"].max()

print(min_year, max_year)

In [ ]:
df.groupby(["cidade", "nome_estacao"]).size()

In [ ]:
df["poluente"].unique()

| Classificação |   PM10   |  PM2.5 |    NO2   |    O3   |    SO2   |
|---------------|:--------:|:------:|:--------:|:-------:|:--------:|
| Muito Bom     |   0-20   |  0-10  |   0-40   |   0-80  |   0-100  |
| Bom           |   21-35  |  11-20 |  41-100  |  81-100 |  101-200 |
| Médio         |   36-50  |  21-25 |  101-200 | 101-180 |  201-350 |
| Fraco         |  51-100  |  26-50 |  201-400 | 181-240 |  351-500 |
| Mau           | 101-1200 | 51-800 | 401-1000 | 241-600 | 501-1250 |

In [ ]:
cols = [
    "cidade",
    "poluente",
    "metrica",
    "nome_estacao",
    "ano",
    "nível_de_poluicao",
    "indice_qualidade_do_ar",
]

poluente = "O3"
poluente_domain = [0, 80]

df_lx = df.query(
    "poluente == @poluente & metrica == 'Média anual' & (nome_estacao == 'Joaquim Magalhães' | nome_estacao == 'Beato')"
).sort_values(by=["ano", "nome_estacao"])[cols]

In [ ]:
df_lx

In [ ]:
# df_lx["ano"].nunique() == (df_lx["ano"].max() - df_lx["ano"].min() + 1)

In [ ]:
# df_lx["ano"] = pd.to_datetime(df_lx["ano"], format="%Y")

In [ ]:
df_lx["maxs"] = df_lx.groupby(["nome_estacao", "poluente"])[
    "nível_de_poluicao"
].transform("max")
df_lx["mins"] = df_lx.groupby(["nome_estacao", "poluente"])[
    "nível_de_poluicao"
].transform("min")
df_lx.head()

In [ ]:
# maxs = df_lx.groupby("nome_estacao", as_index=False).max()
# maxs["queries"] = (
#     "(datum.nome_estacao === '"
#     + maxs["nome_estacao"]
#     + "' && datum.nível_de_poluicao === "
#     + maxs["nível_de_poluicao"].astype(str)
#     + ")"
# )

In [ ]:
# " || ".join(maxs["queries"])

In [ ]:
# mins = df_lx.groupby("nome_estacao", as_index=False).min()
# mins["queries"] = (
#     "(datum.nome_estacao === '"
#     + mins["nome_estacao"]
#     + "' && datum.nível_de_poluicao === "
#     + mins["nível_de_poluicao"].astype(str)
#     + ")"
# )

In [ ]:
# " || ".join(mins["queries"])

In [ ]:
# point_color_query = " || ".join(maxs["queries"]) + " || " + " || ".join(mins["queries"])
# point_color_query

In [ ]:
FONT: str = "Rubik"
FONT_SIZE: int = 12

In [ ]:
base = alt.Chart(df_lx).encode(
    x=alt.X("ano:O", axis=alt.Axis(title=None, grid=True)),
    y=alt.Y(
        "nível_de_poluicao:Q",
        axis=alt.Axis(title=None),
        scale=alt.Scale(domain=poluente_domain),
        impute=alt.ImputeParams(
            value=None, keyvals=list(range(min_year, max_year + 1))
        ),
    ),
    color=alt.Color("nome_estacao:N"),
)

line = base.mark_line()

point = base.mark_circle(tooltip=True, size=30).encode(
    color=alt.condition(
        # point_color_query,
        (alt.datum["nível_de_poluicao"] == alt.datum["maxs"])
        | (alt.datum["nível_de_poluicao"] == alt.datum["mins"]),
        alt.value("red"),
        alt.value("blue"),
    )
)

(line + point).configure_axis(labelFont=FONT, labelFontSize=FONT_SIZE).properties(
    width=300, height=150, title=poluente
).configure_view(strokeWidth=0).configure_legend(
    labelFont=FONT, labelFontSize=FONT_SIZE
)

---